### Actuarial Project

#### Phase One: Data Prep

In [ ]:
# Importing Python packages
import pandas as pd                                              # For data manipulation and analysis
import numpy as np                                               # For numerical operations
import seaborn as sns                                            # For data visualization
import matplotlib.pyplot as plt                                  # For creating visualizations

# To ignore warnings
import warnings                                                  # To handle warnings
warnings.filterwarnings("ignore")                                # Suppress all warnings

In [ ]:
# Import dataset
cleaned_insurance_data = pd.read_csv('/content/drive/MyDrive/Actuarial Project/actuarial_insurance_data.csv')

cleaned_insurance_data.head()

,Age,Is_Senior,Marital_Status,Married_Premium_Discount,Prior_Insurance,Prior_Insurance_Premium_Adjustment,Claims_Frequency,Claims_Severity,Claims_Adjustment,Policy_Type,...,Quotes_Requested,Time_to_Conversion,Credit_Score,Premium_Adjustment_Credit,Region,Premium_Adjustment_Region,Risk_Tier,Loyalty_Band,Engagement_Score,Engagement_Level
0,47,Non-Senior,Married,86,1-5 years,50,0,Low,0,Full Coverage,...,2,99,704,-50,Suburban,50,Low Risk,1-5 Years,13,Medium Engagement
1,37,Non-Senior,Married,86,1-5 years,50,0,Low,0,Full Coverage,...,2,99,726,-50,Urban,100,Low Risk,1-5 Years,13,Medium Engagement
2,49,Non-Senior,Married,86,1-5 years,50,1,Low,50,Full Coverage,...,1,99,772,-50,Urban,100,Low Risk,1-5 Years,15,Medium Engagement
3,62,Senior,Married,86,>5 years,0,1,Low,50,Full Coverage,...,2,2,809,-50,Urban,100,Low Risk,>5 Years,16,High Engagement
4,36,Non-Senior,Single,0,>5 years,0,2,Low,100,Full Coverage,...,2,10,662,50,Suburban,50,Medium Risk,>5 Years,22,High Engagement


### Data Preprocessing

#### Is_Senior

In [ ]:
# Encode 'Is_Senior' as binary (1 = Senior, 0 = Non-Senior)
cleaned_insurance_data['Is_Senior'] = (
    cleaned_insurance_data['Is_Senior']
    .astype(str)              # ensure string
    .str.strip()              # remove spaces
    .str.lower()              # case-insensitive matching
    .map({'senior': 1, 'non-senior': 0})  # map values
    .fillna(0)                # handle any missing or unrecognized values
    .astype(int)              # final type as integer
)

In [ ]:
# Verifying changes to 'Is_Senior' column
cleaned_insurance_data['Is_Senior'].value_counts()


,count
Is_Senior,
0,8407
1,1593


#### Marital_Status

In [ ]:
# Encode Marital_Status: 1 = Married, 0 = Single
cleaned_insurance_data['Marital_Status'] = (
    cleaned_insurance_data['Marital_Status']
    .astype(str)
    .str.strip()
    .str.lower()
    .map({'married': 1, 'single': 0})
    .fillna(0)
    .astype(int)
)


In [ ]:
# Verifying changes to 'Marital_Status' column
cleaned_insurance_data['Marital_Status'].value_counts()

,count
Marital_Status,
0,5101
1,4899


#### Prior_Insurance

In [ ]:
# Encode Prior_Insurance as ordered numeric
prior_order = ['<1 year', '1-5 years', '>5 years']

cleaned_insurance_data['Prior_Insurance'] = (
    pd.Categorical(cleaned_insurance_data['Prior_Insurance'],
                   categories=prior_order,
                   ordered=True)
    .codes
)

In [ ]:
# Verify encoding changes to 'Prior_Insurance' column
cleaned_insurance_data['Prior_Insurance'].value_counts().sort_index()

,count
Prior_Insurance,
0,2134
1,5257
2,2609


#### Prior_Insurance_Premium_Adjustment

In [ ]:
# Coerce to numeric, impute missing with 0, and validate allowed set
allowed = {0, 50, 100}

cleaned_insurance_data['Prior_Insurance_Premium_Adjustment'] = (
    cleaned_insurance_data['Prior_Insurance_Premium_Adjustment']
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
    .astype(int)
)

In [ ]:
# Verifying values in 'Prior_Insurance_Premium_Adjustment' column
cleaned_insurance_data['Prior_Insurance_Premium_Adjustment'].value_counts()

,count
Prior_Insurance_Premium_Adjustment,
50,5257
0,2609
100,2134


#### Claims_Frequency

In [ ]:
# Coerce to integer count, fill missing with 0, and ensure non-negative
cleaned_insurance_data['Claims_Frequency'] = (
    cleaned_insurance_data['Claims_Frequency']
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
)

# Clip negatives (if any data quirks)
cleaned_insurance_data.loc[cleaned_insurance_data['Claims_Frequency'] < 0, 'Claims_Frequency'] = 0

# Round to nearest int (in case any floats slipped in), then cast to int
cleaned_insurance_data['Claims_Frequency'] = (
    cleaned_insurance_data['Claims_Frequency'].round().astype(int)
)

In [ ]:
# Verifying values in 'Claims_Frequency' column
cleaned_insurance_data['Claims_Frequency'].value_counts()

,count
Claims_Frequency,
0,6126
1,2965
2,745
3,141
4,21
5,2


#### Claims_Severity

In [ ]:
# Step 1: Standardise text formatting
cleaned_insurance_data['Claims_Severity'] = (
    cleaned_insurance_data['Claims_Severity']
    .astype(str)
    .str.strip()
    .str.lower()
)

# Step 2: Define the correct order for encoding
severity_order = ['low', 'medium', 'high']

# Step 3: Encode as ordered numeric (0=Low, 1=Medium, 2=High)
cleaned_insurance_data['Claims_Severity'] = (
    pd.Categorical(
        cleaned_insurance_data['Claims_Severity'],
        categories=severity_order,
        ordered=True
    ).codes
)


In [ ]:
# Verifying values in 'Claims_Severity' column
cleaned_insurance_data['Claims_Severity'].value_counts()

,count
Claims_Severity,
0,7003
1,2038
2,959


#### Claims_Adjustment

In [ ]:
# Coerce numeric, impute missing (0 is a reasonable neutral for an adjustment)
cleaned_insurance_data['Claims_Adjustment'] = (
    cleaned_insurance_data['Claims_Adjustment']
    .apply(pd.to_numeric, errors='coerce')
    .fillna(0)
)

In [ ]:
# Verifying values in 'Claims_Adjustment' column
cleaned_insurance_data['Claims_Adjustment'].value_counts()

,count
Claims_Adjustment,
0,6126
50,2096
100,1131
200,428
150,92
400,75
300,34
600,15
250,2


#### Policy_Type

In [ ]:
# Encode Policy_Type: 1 = Full Coverage, 0 = Liability-Only
cleaned_insurance_data['Policy_Type'] = (
    cleaned_insurance_data['Policy_Type']
    .astype(str)
    .str.strip()
    .str.lower()
    .map({'full coverage': 1, 'liability-only': 0})
    .fillna(0)
    .astype(int)
)

In [ ]:
# Verifying values in 'Policy_Type' column
cleaned_insurance_data['Policy_Type'].value_counts()

,count
Policy_Type,
1,6007
0,3993


#### Source_of_Lead

This is nominal (no natural order). For GLM we must create one-hot dummies and drop a baseline (to avoid the dummy trap). Because this has >2 levels, overwriting a single column with numbers would imply an artificial order — not good. Best practice is to expand to dummies and drop the original.

In [ ]:
# Make two dummy columns (baseline = Online)
cleaned_insurance_data['Source_of_Lead_Agent']    = (cleaned_insurance_data['Source_of_Lead'] == 'Agent').astype(int)
cleaned_insurance_data['Source_of_Lead_Referral'] = (cleaned_insurance_data['Source_of_Lead'] == 'Referral').astype(int)

# --- Minimal verification ---

# 1) Unique patterns → should be only (0,0)=Online, (1,0)=Agent, (0,1)=Referral
print(cleaned_insurance_data[['Source_of_Lead_Agent','Source_of_Lead_Referral']].drop_duplicates())

# 2) Counts from original vs reconstructed from dummies
print(cleaned_insurance_data['Source_of_Lead'].value_counts())

    Source_of_Lead_Agent  Source_of_Lead_Referral
0                      1                        0
1                      0                        0
11                     0                        1
Source_of_Lead
Online      6035
Agent       3004
Referral     961
Name: count, dtype: int64


#### Conversion_Status

In [ ]:
# Conversion_Status: 1 = Converted, 0 = Not Converted
cleaned_insurance_data['Conversion_Status'] = (cleaned_insurance_data['Conversion_Status'] == 'Converted').astype(int)


In [ ]:
# Verifying values in 'Conversion_Status' column
cleaned_insurance_data['Conversion_Status'].value_counts()

,count
Conversion_Status,
1,5767
0,4233


#### Region

In [ ]:
# Region: baseline = Rural
cleaned_insurance_data['Region_Urban']    = (cleaned_insurance_data['Region'] == 'Urban').astype(int)
cleaned_insurance_data['Region_Suburban'] = (cleaned_insurance_data['Region'] == 'Suburban').astype(int)

In [ ]:
print("Region flags:", cleaned_insurance_data[['Region','Region_Urban','Region_Suburban']].drop_duplicates().head())

Region flags:      Region  Region_Urban  Region_Suburban
0  Suburban             0                1
1     Urban             1                0
5     Rural             0                0


#### Risk_Tier

In [ ]:
# Risk_Tier: Low < Medium < High  → 0/1/2
risk_order = ['Low Risk','Medium Risk','High Risk']
cleaned_insurance_data['Risk_Tier'] = pd.Categorical(cleaned_insurance_data['Risk_Tier'], categories=risk_order, ordered=True).codes

In [ ]:
# Verifying values in 'Risk_Tier' column
cleaned_insurance_data['Risk_Tier'].value_counts()

,count
Risk_Tier,
0,6358
1,2535
2,1107


#### Loyalty_Band

In [ ]:
# Loyalty_Band: New < 1-5 < >5 → 0/1/2
loyalty_order = ['New Customer','1-5 Years','>5 Years']
cleaned_insurance_data['Loyalty_Band'] = pd.Categorical(cleaned_insurance_data['Loyalty_Band'], categories=loyalty_order, ordered=True).codes


In [ ]:
# Verifying values in 'Loyalty_Band' column
cleaned_insurance_data['Loyalty_Band'].value_counts()

,count
Loyalty_Band,
1,5257
2,2609
0,2134


#### Engagement_Level

In [ ]:
# Engagement_Level: Low < Medium < High → 0/1/2
eng_order = ['Low Engagement','Medium Engagement','High Engagement']
cleaned_insurance_data['Engagement_Level'] = pd.Categorical(cleaned_insurance_data['Engagement_Level'], categories=eng_order, ordered=True).codes

In [ ]:
# Verifying values in 'Engagement_Level' column
cleaned_insurance_data['Engagement_Level'].value_counts()

,count
Engagement_Level,
1,5557
2,4383
0,60


#### Verifying Checks on the cleaned dataset.

In [ ]:
cleaned_insurance_data.head()

,Age,Is_Senior,Marital_Status,Married_Premium_Discount,Prior_Insurance,Prior_Insurance_Premium_Adjustment,Claims_Frequency,Claims_Severity,Claims_Adjustment,Policy_Type,...,Region,Premium_Adjustment_Region,Risk_Tier,Loyalty_Band,Engagement_Score,Engagement_Level,Source_of_Lead_Agent,Source_of_Lead_Referral,Region_Urban,Region_Suburban
0,47,0,1,86,1,50,0,0,0,1,...,Suburban,50,0,1,13,1,1,0,0,1
1,37,0,1,86,1,50,0,0,0,1,...,Urban,100,0,1,13,1,0,0,1,0
2,49,0,1,86,1,50,1,0,50,1,...,Urban,100,0,1,15,1,0,0,1,0
3,62,1,1,86,2,0,1,0,50,1,...,Urban,100,0,2,16,2,0,0,1,0
4,36,0,0,0,2,0,2,0,100,1,...,Suburban,50,1,2,22,2,1,0,0,1


In [ ]:
# Checking the shape of the data
cleaned_insurance_data.shape

(10000, 35)

In [ ]:
# Checking basic info about the data
cleaned_insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 35 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   Age                                 10000 non-null  int64 
 1   Is_Senior                           10000 non-null  int64 
 2   Marital_Status                      10000 non-null  int64 
 3   Married_Premium_Discount            10000 non-null  int64 
 4   Prior_Insurance                     10000 non-null  int8  
 5   Prior_Insurance_Premium_Adjustment  10000 non-null  int64 
 6   Claims_Frequency                    10000 non-null  int64 
 7   Claims_Severity                     10000 non-null  int8  
 8   Claims_Adjustment                   10000 non-null  int64 
 9   Policy_Type                         10000 non-null  int64 
 10  Policy_Adjustment                   10000 non-null  int64 
 11  Premium_Amount                      10000 non-null  int

#### Save Cleaned Dataset for GLM

In [ ]:
# Save cleaned dataset as a new CSV
cleaned_insurance_data.to_csv("model_ready_insurance_data.csv", index=False)
